<a href="https://colab.research.google.com/github/priya369-ps/git-demo/blob/main/notebooks/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3



In [ ]:
from google.colab import files
uploaded = files.upload()



Saving restaurants.sql to restaurants (1).sql
Saving users.json to users.json
Saving orders.csv to orders (1).csv


In [ ]:
os.listdir()


['.config',
 'orders.csv',
 'restaurants (1).sql',
 'users.json',
 'database.db',
 'restaurants.sql',
 'orders (1).csv',
 'sample_data']

In [131]:
df_orders = pd.read_csv('orders.csv')
df_users=pd.read_json('users.json')
conn = sqlite3.connect(':memory:')
with open('restaurants.sql', 'r') as f:
    sql_script = f.read()
conn.executescript(sql_script)
df_restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
conn.close()

In [123]:
df_orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [129]:
df_restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [132]:
df_users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [134]:
print("Orders CSV:", df_orders.shape)
print("Users JSON:", df_users.shape)
print("Restaurants SQL:",df_restaurants.shape)


Orders CSV: (10000, 6)
Users JSON: (3000, 4)
Restaurants SQL: (500, 4)


In [135]:
df_orders['order_date'] = pd.to_datetime(df_orders['order_date'], dayfirst=True)

In [137]:
df_master = df_orders.merge(df_users, on='user_id', how='left')
df_master = df_master.merge(df_restaurants, on='restaurant_id', how='left', suffixes=('', '_rest'))


In [174]:
#1The city with the highest revenue from Gold members
df_merged = pd.merge(df_orders, df_users, on='user_id')
gold_revenue = df_merged[df_merged['membership'] == 'Gold'].groupby('city')['total_amount'].sum()
top_city = gold_revenue.idxmax()
total_value = gold_revenue[top_city]
print(top_city)
print(total_value)


Chennai
1080909.79


In [144]:
# 2. Cuisine with highest average order value
cuisine_avg = df_master.groupby('cuisine')['total_amount'].mean()
top_cuisine = cuisine_avg.idxmax()
top_cuisine

'Mexican'

In [147]:
# 3. Distinct users with total spend > 1000
user_total_spend = df_master.groupby('user_id')['total_amount'].sum()
high_spend_count = (user_total_spend > 1000).sum()
print(high_spend_count)

2544


In [142]:
#4 restaurant rating range generated the highest total revenue
def get_rating_range(r):
    if 3.0 <= r <= 3.5: return '3.0 – 3.5'
    if 3.6 <= r <= 4.0: return '3.6 – 4.0'
    if 4.1 <= r <= 4.5: return '4.1 – 4.5'
    if 4.6 <= r <= 5.0: return '4.6 – 5.0'
    return 'Other'

df_master['rating_range'] = df_master['rating'].apply(get_rating_range)
rating_rev = df_master.groupby('rating_range')['total_amount'].sum()
top_rating_range = rating_rev.idxmax()
print(top_rating_range)

4.6 – 5.0


In [151]:
# 5. Gold members highest average order value city
gold_city_avg = df_master[df_master['membership'] == 'Gold'].groupby('city')['total_amount'].mean()
top_gold_avg_value_city = gold_city_avg.idxmax()
print(top_gold_avg_value_city)



Chennai


In [152]:
# 6. Cuisine with lowest number of distinct restaurants
cuisine_rest_counts = df_master.groupby('cuisine')['restaurant_id'].nunique()
cuisine = cuisine_rest_counts.idxmin()
print(cuisine)



Chinese


In [156]:
# 7. Percentage of total orders by Gold members
gold_percent = round((len(df_master[df_master['membership'] == 'Gold']) / len(df_master)) * 100)
gold_percent



50

In [166]:
# 8. Restaurant with highest AOV but < 20 total orders
restaurant_list = [
    "Grand Cafe Punjabi", "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine", "Ruchi Foods Chinese"
]
stats = df_merged[df_merged["restaurant_name"].isin(restaurant_list)].groupby("restaurant_name")["total_amount"].agg(['count', 'mean'])

top_restaurant_name = stats[stats['count'] < 20]['mean'].idxmax()
top_value = stats.loc[top_restaurant_name, 'mean']

print(top_value)




686.6031578947368


In [164]:
# 9. Membership + Cuisine combination with highest revenue
combo_rev = df_master.groupby(['membership', 'cuisine'])['total_amount'].sum()
high_rev_combo = combo_rev.idxmax()
high_rev_combo


('Regular', 'Mexican')

In [172]:
# 9. Membership + Cuisine combination with highest revenue
options = [('Gold', 'Italian'), ('Gold', 'Indian'), ('Regular', 'Indian'), ('Regular', 'Chinese')]
stats = df_master.groupby(['membership', 'cuisine'])['total_amount'].sum()
combo = stats[stats.index.isin(options)].idxmax()
print( combo[0] + combo[1])

GoldItalian


In [168]:
# 10. Quarter with highest revenue
df_master['quarter'] = df_master['order_date'].dt.to_period('Q')
quarter_rev = df_master.groupby('quarter')['total_amount'].sum()
top_quarter = quarter_rev.idxmax()
top_quarter

Period('2023Q3', 'Q-DEC')

###FILL IN THE BLANKS

In [121]:
final_merged['total_amount'] = pd.to_numeric(final_merged['total_amount'], errors='coerce')
final_merged['rating'] = pd.to_numeric(final_merged['rating'], errors='coerce')
for col in ['membership', 'city', 'cuisine']:
    final_merged[col] = final_merged[col].astype(str).str.strip()


In [105]:
total_gold_orders = final_merged[final_merged['membership'] == 'Gold'].shape[0]
total_gold_orders


4987

In [106]:
total_hyderabad_revenue = round(final_merged[final_merged['city'].str.lower() == 'hyderabad']['total_amount'].sum())
total_hyderabad_revenue


1889367

In [108]:
distinct_users = final_merged['user_id'].nunique()
distinct_users

2883

In [117]:
avg_order_gold = round(final_merged[final_merged['membership'] == 'Gold']['total_amount'].mean(), 2)
print(avg_order_gold)


797.15


In [120]:
import pandas as pd
import sqlite3
df_orders = pd.read_csv('orders.csv')
conn = sqlite3.connect(':memory:')
with open('restaurants.sql', 'r') as f:
    sql_script = f.read()
conn.executescript(sql_script)
df_restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
conn.close()

In [118]:
df_merged = df_orders.merge(df_restaurants[['restaurant_id', 'rating']], on='restaurant_id', how='left')
high_rated_orders = df_merged[df_merged['rating'] >= 4.5]
total_high_rated_orders = len(high_rated_orders)
print(total_high_rated_orders)

3374


In [116]:
gold_members = final_merged[final_merged['membership'] == 'Gold']
city_revenue = gold_members.groupby('city')['total_amount'].sum().reset_index()
top_city = city_revenue.sort_values('total_amount', ascending=False).iloc[0]['city']
orders_top_city = gold_members[gold_members['city'] == top_city].shape[0]
print(orders_top_city)


1337
